In [2]:
import tensorflow as tf
from tensorboard.backend.event_processing import event_accumulator
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

def create_multi_scalar_animation(logdir, output_file='training_metrics_fixed.gif'):
    ea = event_accumulator.EventAccumulator(logdir)
    ea.Reload()
    
    scalar_tags = [
        'eval/mean_ep_length', 'eval/mean_reward',
        'train/approx_kl', 'train/clip_fraction',
        'train/clip_range', 'train/entropy_loss',
        'train/explained_variance', 'train/learning_rate',
        'train/loss', 'train/policy_gradient_loss',
        'train/value_loss'
    ]
    
    fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(18, 16))
    fig.patch.set_facecolor('#1a1a1a')
    plt.style.use('dark_background')

    # Flatten axes for easier handling
    axes = axes.flatten()
    all_data = {}
    lines = {}

    for idx, tag in enumerate(scalar_tags):
        ax = axes[idx]
        ax.set_facecolor('#262626')
        ax.grid(True, linestyle='--', alpha=0.4, color='#888888')

        events = ea.Scalars(tag)
        steps = [event.step for event in events]
        values = [event.value for event in events]
        all_data[tag] = {'steps': steps, 'values': values}

        line, = ax.plot([], [], lw=2, color='#00ff99')
        lines[tag] = line

        ax.set_xlim(min(steps), max(steps))
        ax.set_ylim(
            min(values) - 0.1 * abs(min(values)), 
            max(values) + 0.1 * abs(max(values))
        )

        title = tag.replace('train/', '').replace('eval/', '').replace('_', ' ').title()
        ax.set_title(title, color='#00ff99', fontsize=10, pad=8)
        ax.tick_params(colors='#ffffff', labelsize=8)

        if idx % 3 == 0:
            ax.set_ylabel('Value', color='#ffffff', fontsize=10)
        if idx >= len(scalar_tags) - 3:
            ax.set_xlabel('Steps', color='#ffffff', fontsize=10)

    # Turn off unused axes
    for ax in axes[len(scalar_tags):]:
        ax.axis('off')

    # Adjust layout to prevent title overlap
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)  # Add space for the main title
    fig.suptitle('Training Metrics Overview', 
                 color='#00ff99', 
                 fontsize=18, 
                 y=0.98)  # Position title slightly higher
    
    def init():
        for line in lines.values():
            line.set_data([], [])
        return list(lines.values())

    def animate(frame):
        for tag, line in lines.items():
            steps = all_data[tag]['steps'][:frame]
            values = all_data[tag]['values'][:frame]
            line.set_data(steps, values)
        return list(lines.values())

    anim = animation.FuncAnimation(
        fig, 
        animate, 
        init_func=init,
        frames=min(len(all_data[tag]['steps']) for tag in scalar_tags),
        interval=500,
        blit=True,
        repeat=True
    )

    anim.save(
        output_file,
        writer='pillow',
        fps=10,
        dpi=100
    )
    plt.close()


run_name_to_plot = "PPO_6"
logdir = f"/home/tommy/Downloads/ai-playground/tensorboard_logs/{run_name_to_plot}"
create_multi_scalar_animation(logdir)